In [1]:
# what biolink:category does my identifier represent
# how to find the predicates used for gene to disease mappings


In [14]:
from linkml_runtime.utils.schemaview import SchemaView
import requests 
from pprint import pprint
# note you can also use a path on a local filesystem
view = SchemaView("https://raw.githubusercontent.com/biolink/biolink-model/master/biolink-model.yaml")

In [15]:
# what biolink:category does my identifier represent?
# id_prefixes
# mappings

element = view.get_element('disease')
print(type(element))

element = view.get_element('phenotype of')
print(element.name)


In [16]:
# find inverses of a predicate
print("inverse is: " + view.inverse(element.name))

In [17]:
# id_prefixes
prefixed_categories = view.get_elements_applicable_by_identifier("DOID:4")
print(prefixed_categories)

In [18]:
# mappings 

mapped_categories = view.get_category_by_mapping('SO:0001583')
print(mapped_categories)


In [21]:
# object = 'gene'
# object = 'disease'
# object = 'phenotypic feature'
object = 'sequence variant'

query_prefix = f'https://www.ebi.ac.uk/ols/api/ontologies/_ontology/terms/'
mappings = view.get_mappings(object)
if len(mappings) == 0:
    print("no exact mappings found for: " + object)
for exact_mapping in mappings.get('exact'):
    url = query_prefix.replace("_ontology", exact_mapping.split(":")[0]) + "http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252F" + exact_mapping.replace(":", "_")
    response = requests.get(url)
    if response.status_code == 200:
        print(exact_mapping)
        element = view.get_element(object)
        if element.description is None:
            print("biolink description is empty")
        else:
            print("description: " + element.description +"\n")
        if response.json().get('description') is None:
            print ("no definition found via OLS")
        else: 
            for description in response.json().get('description'):
                print("OLS description: " + description)
    else:
        print(exact_mapping + ": can't find any matching terms in OLS that don't return 404 errors")
     

In [10]:
# is my element a mixin?

e = view.get_element('gene or gene product')
view.is_mixin(e.name)

In [22]:
# view poly hierarchy - a gene is a chemical and biological entity

ancestors = view.class_ancestors('gene')
for a in ancestors:
    print(a)


In [23]:
# how to find the predicates used for gene to disease mappings
# association:   
#  exact_mappings:
#      - OBAN:association
#      - rdf:Statement
#      - owl:Axiom
# check out the biolink-model onion again

associations = [c for c in view.all_classes().keys() if view.is_relationship(c)][0:100]
print(len(associations))
for a in associations:
    if a.startswith('gene'):
        print(a)


In [26]:
for association in associations:
    domain_element = view.get_element(view.induced_slot('subject', association).range)
    range_element = view.get_element(view.induced_slot('object', association).range)
    
    if 'gene' in view.class_ancestors(domain_element.name) and 'disease' in view.class_ancestors(range_element.name):
        print(association)
        #print(view.induced_slot('subject', association))
    if 'gene or gene product' in view.class_ancestors(domain_element.name) and 'disease' in view.class_ancestors(range_element.name):
        print(association)
        print(view.induced_slot('subject', association))


In [ ]:
# find predicates for those associations
# at this point, navigating the online doc might be easiest if you just want answers. 
# programatically, we can get the predicates that have equivalent domain and range constraints to find which 
# coudl be used for associations above.
